Here we try to reproduce the assembly documentation from this [best practices example](https://github.com/SynBioDex/SBOL-examples/blob/main/SBOL/best-practices/BP011/examples.md)



In [26]:
import sbol3
import tyto
import Bio.Restriction
from pydna.dseq import Dseq


First, prepare the SBOL3 workspace and the document that we will be working with

In [17]:
# Prepare Document
sbol3.set_namespace('http://https://shareyourcloning.netlify.app//testfiles')
doc = sbol3.Document()

Next, create the input objects, a circular plasmid and a linear fragment to be digested for insertion.

For the sequences, we'll use notional base values for everything but the restriction enzyme sites themselves.

In [18]:
# First, create the input sequences
plasmid_bases = 'a'*36 + Bio.Restriction.SalI.site + 't'*20 + Bio.Restriction.AscI.site + 'a'*2000
plasmid_seq = doc.add(sbol3.Sequence('plasmid_seq' , elements=plasmid_bases, encoding=sbol3.IUPAC_DNA_ENCODING))

pcr_prod_bases = 'a'*7 + Bio.Restriction.SalI.site + 't'*2244 + Bio.Restriction.AscI.site + 'g'*10
pcr_prod_seq = doc.add(sbol3.Sequence('pcr_prod_seq' , elements=pcr_prod_bases, encoding=sbol3.IUPAC_DNA_ENCODING))

# Next, ceate the Components, marking the restriction sites on each
salI_plasmid = sbol3.SequenceFeature(sbol3.Range(plasmid_seq, 37, 42), roles=[tyto.SO.restriction_enzyme_recognition_site], name='salI_on_plasmid')
ascI_plasmid = sbol3.SequenceFeature(sbol3.Range(plasmid_seq, 62, 69), roles=[tyto.SO.restriction_enzyme_recognition_site], name='ascI_on_plasmid')

salI_pcr_prod = sbol3.SequenceFeature(sbol3.Range(pcr_prod_seq, 7, 12), roles=[tyto.SO.restriction_enzyme_recognition_site], name='salI_on_pcr_prod')
ascI_pcr_prod = sbol3.SequenceFeature(sbol3.Range(pcr_prod_seq, 2256, 2263), roles=[tyto.SO.restriction_enzyme_recognition_site], name='ascI_on_pcr_prod')

# These are the two actual inputs to the assembly: the plasmid is circular DNA, while the PCR product is linear
plasmid = doc.add(sbol3.Component('plasmid', [sbol3.SBO_DNA, sbol3.SO_CIRCULAR], features=[salI_plasmid, ascI_plasmid], sequences=[plasmid_seq], 
                                  description='Plasmid to be opened for insertion'))
pcr_prod = doc.add(sbol3.Component('pcr_prod', sbol3.SBO_DNA, features=[salI_pcr_prod, ascI_pcr_prod], sequences=[pcr_prod_seq], 
                                   description='PCR product input containing insert'))

Next, we will create the assembly plan itself, which is where all of the reactions will be described.
The final product will be generated by the assembly plan, but will not actually _be_ the assembly plan itself, per BP011

In [59]:
plan = doc.add(sbol3.Component('my_assembly_plan', sbol3.SBO_FUNCTIONAL_ENTITY, description='Plan to use SalI and AscI for Type II assembly of plasmid and pcr_prod'))

For the digestion, we'll need the two actual restriction enzymes.
Since we don't need to know their full details, they are represented as externally defined features

In [60]:
salI_enzyme = sbol3.ExternallyDefined(sbol3.SBO_PROTEIN, name='SalI', definition='http://purl.obolibrary.org/obo/PR_Q53608', description='SalI restriction enzyme')
ascI_enzyme = sbol3.ExternallyDefined(sbol3.SBO_PROTEIN, name='AscI', definition='http://purl.obolibrary.org/obo/PR_E3VXA3', description='AscI restriction enzyme')
plan.features.extend([salI_enzyme, ascI_enzyme])

Note that by setting the enzyme names to their standard names, we can use the names to look up enzymes in BioPython and use them to cut in pyDNA.
The fragments of such a product are what we will need to represent in a digestion plan, in order for the user to pick between them.

In [61]:
dseq = Dseq(pcr_prod_seq.elements)
fragments = dseq.cut(Bio.Restriction.__dict__[salI_enzyme.name])
print(fragments)

(Dseq(-12)
aaaaaaaG
tttttttCAGCT, Dseq(-2267)
TCGACttt..gggg
    Gaaa..cccc)


As a simple starting point, Let us describe the reaction to digest the linear PCR product with only SalI

In [62]:
pcr_prod_subcomp = sbol3.SubComponent(pcr_prod)
plan.features.append(pcr_prod_subcomp)

# There are two outputs that we will need to describe, in order to select between: the left side of the cut and the right side of the cut
# These will start out as generic LocalSubComponent objects, which can then be filled in
pcr_product_left_of_SalI = sbol3.LocalSubComponent(sbol3.SBO_DNA, name="Left of SalI")
pcr_product_right_of_SalI = sbol3.LocalSubComponent(sbol3.SBO_DNA, name="Right of SalI")

# They are ordered before and after the salI cut site, which we point to within its
salI_pcr_prod_ref = sbol3.ComponentReference(in_child_of=pcr_prod_subcomp, refers_to=salI_pcr_prod)
# Note that the features need to be added to the plan before we start making references to them 
plan.features.extend([pcr_product_left_of_SalI, pcr_product_right_of_SalI, salI_pcr_prod_ref])

# Sequence ordering constrains express the "left" and "right" concepts
# Note that we use "precedes" rather than "strictly precedes" because part of the restriction site ends up in the product
plan.constraints.extend([
    sbol3.Constraint(restriction=sbol3.SBOL_PRECEDES, subject=pcr_product_left_of_SalI, object=salI_pcr_prod_ref),
    sbol3.Constraint(restriction=sbol3.SBOL_PRECEDES, subject=salI_pcr_prod_ref, object=pcr_product_right_of_SalI),
])

# The reaction then states that the enzyme modifies the pcr product to produce the left and right digests
salI_pcr_digestion = sbol3.Interaction(tyto.SBO.cleavage, name='plasmid_digestion', participations=[
    sbol3.Participation(tyto.SBO.modifier, salI_enzyme),
    sbol3.Participation(tyto.SBO.reactant, pcr_prod_subcomp),
    sbol3.Participation(tyto.SBO.product, pcr_product_left_of_SalI),
    sbol3.Participation(tyto.SBO.product, pcr_product_right_of_SalI)
])
plan.interactions.append(salI_pcr_digestion)

At this point, if we wanted we could use the ordering relationships to align the outputs of the digestion with the fragments from the pyDNA cut

In [63]:
import sbol_utilities.calculate_sequences

# I'm just kludging the sorting function here for the moment. It should be done as a generalization of sbol_utilities.calculate_sequences.order_subcomponents
ordered_elements = \
    [c.subject.lookup() for c in plan.constraints if c.restriction == sbol3.SBOL_PRECEDES and c.object.lookup() == salI_pcr_prod_ref] + \
    [c.object.lookup() for c in plan.constraints if c.restriction == sbol3.SBOL_PRECEDES and c.subject.lookup() == salI_pcr_prod_ref]

# The ordering gives a deterministic result:
assert(ordered_elements == [pcr_product_left_of_SalI, pcr_product_right_of_SalI])

# This can then be aligned with the fragments:
[(c.name, f) for c,f in zip(ordered_elements, fragments)]

[('Left of SalI',
  Dseq(-12)
  aaaaaaaG
  tttttttCAGCT),
 ('Right of SalI',
  Dseq(-2267)
  TCGACttt..gggg
      Gaaa..cccc)]

We now have a representation of the full reaction without having selected whether the left or right side of the reaction is what we want for the output or actually needing to compute a sequence.

To actually select one of the two side as the intended product of the digestion, we add an identity constraint that links the output component to the desired product, left or right.

We wouldn't need this identity constraint if we knew which element we wanted ahead of time or chose to rewrite the plan after choosing: we could just have the product by part of the reaction. Using the identity constraint, however, lets the plan explicitly represent the choice between left and right and the ability to change that choice.

In [64]:
restricted_pcr_prod = sbol3.Component('restricted_pcr_prod', sbol3.SBO_DNA, description='restricted_pcr_prod')
restricted_pcr_prod_subcomp = sbol3.SubComponent(restricted_pcr_prod)
plan.features.append(restricted_pcr_prod_subcomp)

plan.constraints.append(sbol3.Constraint(restriction=sbol3.SBOL_VERIFY_IDENTICAL, subject=pcr_product_left_of_SalI, object=restricted_pcr_prod_subcomp))

We can also explicitly mark the inputs and outputs of the plan:

In [ ]:
plan.interface = sbol3.Interface(inputs=[pcr_prod_subcomp], outputs=[restricted_pcr_prod_subcomp])

Adding a second cut site or moving to a circular plasmid will mean adding another cut site reference an adding more constraints and LocalSubComponent objects.

The plasmid digestion and ligation can either be done within the same plan by adding more reactions or can be added as separate plans, chaining together outputs of the digestion plans as inputs of the ligation plan.

# _Below this point is other prior code that hasn't been revised into the current example_

In [3]:
final_prod_seq = sbol3.Sequence('final_prod_seq' , elements='aaaaaaaaaa', encoding=sbol3.IUPAC_DNA_ENCODING)

restricted_plasmid_seq = sbol3.Sequence('restricted_plasmid_seq' , elements='aaaaaaaaaa', encoding=sbol3.IUPAC_DNA_ENCODING)
restricted_pcr_prod_seq = sbol3.Sequence('restricted_pcr_prod_seq' , elements='aaaaaaaaaa', encoding=sbol3.IUPAC_DNA_ENCODING)

# These two are the intermediate results from digesting the inputs
restricted_plasmid = sbol3.Component('restricted_plasmid', sbol3.SBO_DNA, sequences=[restricted_plasmid_seq], description='restricted_plasmid')
# This is the final result from ligating the two intermediate products
final_prod = sbol3.Component('final_prod', sbol3.SBO_DNA, sequences=[final_prod_seq], description='final_prod')

plasmid_subcomp = sbol3.SubComponent(plasmid)
final_prod_subcomp = sbol3.SubComponent(final_prod)
restricted_plasmid_subcomp = sbol3.SubComponent(restricted_plasmid)

final_prod.features.extend([plasmid_subcomp, pcr_prod_subcomp, final_prod_subcomp, restricted_plasmid_subcomp, restricted_pcr_prod_subcomp])


In [113]:
# Digestion interactions


salI_participation_plasmid = sbol3.Participation(tyto.SBO.modifier, salI_subcomp)
ascI_participation_plasmid = sbol3.Participation(tyto.SBO.modifier, ascI_subcomp)
plasmid_reactant_participation = sbol3.Participation(sbol3.SBO_REACTANT, plasmid_subcomp)
restricted_plasmid_product_participation = sbol3.Participation(tyto.SBO.product, restricted_plasmid_subcomp)


plasmid_digestion = sbol3.Interaction(tyto.SBO.cleavage, name='plasmid_digestion', participations=[
    plasmid_reactant_participation,
    restricted_plasmid_product_participation,
    salI_participation_plasmid,
    ascI_participation_plasmid
])

plasmid_digestion_component = sbol3.Component('plasmid_digestion_component', sbol3.SBOL_COMPONENT, interactions=[plasmid_digestion])

# Add a constraint components references to cutsites of interest left_cut --precedes--> right_cut (see figure backbone)
# this sequence feature is in a precedes relationship, or after

salI_participation_pcr_prod = sbol3.Participation(tyto.SBO.modifier, salI_subcomp)
ascI_participation_pcr_prod = sbol3.Participation(tyto.SBO.modifier, ascI_subcomp)
pcr_prod_reaction_participation = sbol3.Participation(sbol3.SBO_REACTANT, pcr_prod_subcomp)
restricted_pcr_prod_product_participation = sbol3.Participation(tyto.SBO.product, restricted_pcr_prod_subcomp)

pcr_prod_digestion = sbol3.Interaction(tyto.SBO.cleavage, name='pcr_prod_digestion', participations=[
    pcr_prod_reaction_participation,
    restricted_pcr_prod_product_participation,
    salI_participation_pcr_prod,
    ascI_participation_pcr_prod
])

pcr_prod_digestion_component = sbol3.Component('pcr_prod_digestion_component', sbol3.SBOL_COMPONENT, interactions=[pcr_prod_digestion])

In [114]:
# Ligation interaction
ligase_subcomp = sbol3.SubComponent('http://purl.obolibrary.org/obo/blah', description='T4 DNA ligase')
pcr_prod.features.append(ligase_subcomp)

ligase_participation = sbol3.Participation(tyto.SBO.modifier, ligase_subcomp)
restricted_plasmid_reactant_participation = sbol3.Participation(sbol3.SBO_REACTANT, restricted_plasmid_subcomp)
restricted_pcr_prod_reactant_participation = sbol3.Participation(sbol3.SBO_REACTANT, restricted_pcr_prod_subcomp)
final_prod_product_participation = sbol3.Participation(tyto.SBO.product, final_prod_subcomp)

ligation = sbol3.Interaction(tyto.SBO.conversion, name='ligation', participations=[
    restricted_plasmid_reactant_participation,
    restricted_pcr_prod_reactant_participation,
    final_prod_product_participation,
    ligase_participation
])

ligation_component = sbol3.Component('ligation_component', sbol3.SBOL_COMPONENT, interactions=[ligation])




In [115]:
doc.add(final_prod)
doc.add(plasmid_digestion_component)
doc.add(pcr_prod_digestion_component)
doc.add(ligation_component)
doc.add(restricted_plasmid)
doc.add(restricted_pcr_prod)

doc.write('example_combine.ttl')
doc.write('example_combine.xml')

!bash make_graph.sh

showing instances: True
showing classes: False
showing namespaces: True
using rdf format: turtle
graphic file created: example_combine.ttl.svg
graphic was created with output file name: example_combine.ttl.svg
